In [ ]:
%pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from rapidfuzz import process, fuzz
import unicodedata

In [ ]:
# Load CSVs
fpl_df = pd.read_csv("beautiful_fpl_data.csv")
all_match_df = pd.read_csv("all_match_data.csv")

# Create simpler match name in all_match_data
all_match_df['simpler-match-name'] = all_match_df['match-name'].apply(lambda x: ' '.join(x.split(' ')[1:]))


In [ ]:
#There are 562 players with non zero minutes in the league, both dfs have 562 players, I am dropping players with zero minutes
print(all_match_df.Player.unique().shape, fpl_df[fpl_df.minutes > 0].name.unique().shape)
fpl_df = fpl_df[fpl_df.name.isin(fpl_df[fpl_df.minutes > 0].name.unique())]
print(all_match_df.Player.unique().shape, fpl_df.name.unique().shape)

(562,) (562,)
(562,) (562,)


In [ ]:
def clean_name(name):
    if isinstance(name, str):
        # Normalize unicode characters (é -> e, ü -> u, ñ -> n, etc.)
        name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
        # Remove remaining special characters
        name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
        # Lowercase and strip
        name = name.lower().strip()
    return name

def clean_match_name(name):
    if isinstance(name, str):
        return name.strip()
    return name

fpl_df['cleaned_name'] = fpl_df['name'].apply(clean_name)
all_match_df['cleaned_Player'] = all_match_df['Player'].apply(clean_name)

fpl_df['cleaned_match_name'] = fpl_df['match-name'].apply(clean_match_name)
all_match_df['cleaned_match_name'] = all_match_df['match-name'].apply(clean_match_name)

fpl_only_cleaned = set(fpl_df['cleaned_name'].unique()) - set(all_match_df['cleaned_Player'].unique())
all_match_only_cleaned = set(all_match_df['cleaned_Player'].unique()) - set(fpl_df['cleaned_name'].unique())

print(f"\n{len(fpl_only_cleaned)} FPL names not in all_match_df")
print(f"{len(all_match_only_cleaned)} all_match_df names not in FPL")

def name_variations(name):
    parts = name.split()
    if len(parts) > 1:
        reversed_name = " ".join(parts[::-1])
        return [name, reversed_name]
    return [name]

player_name_mapping = {}
for fpl_name in fpl_only_cleaned:
    best_score = 0
    best_match = None
    for candidate in all_match_only_cleaned:
        for variant in name_variations(fpl_name):
            score = fuzz.token_sort_ratio(variant, candidate)
            if score > best_score:
                best_score = score
                best_match = candidate
    #if best_score > 80:  # Threshold for fuzzy match
    player_name_mapping[fpl_name] = best_match

print(f"\nGenerated {len(player_name_mapping)} best-guess player name mappings")

# -----------------------------
# 6️⃣ Create a DataFrame for export
# -----------------------------
mapping_df = pd.DataFrame({
    "fpl_cleaned_name": list(player_name_mapping.keys()),
    "all_match_best_guess": list(player_name_mapping.values())
})

# Add a column for manual final choice
mapping_df["chosen_name"] = mapping_df['all_match_best_guess']  # Fill manually later
mapping_df["verified"] = False
mapping_df["all_match_names_reference"] = sorted(list(all_match_only_cleaned))

# Optional: save to CSV for manual review
#mapping_df.to_csv("player_name_mapping_review.csv", index=False)

#print("\nExported mapping DataFrame for manual review (fill 'chosen_name').")
mapping_df.head(20)


109 FPL names not in all_match_df
109 all_match_df names not in FPL

Generated 109 best-guess player name mappings

Exported mapping DataFrame for manual review (fill 'chosen_name').


,fpl_cleaned_name,all_match_best_guess,chosen_name,verified,all_match_names_reference
0,diogo dalot teixeira,diogo dalot,diogo dalot,False,abdul fatawu issahaku
1,joao victor gomes da silva,joao gomes,joao gomes,False,albert grnbaek
2,jefferson lerma solis,jefferson lerma,jefferson lerma,False,alex moreno
3,youssef ramalho chermiti,youssef chermiti,youssef chermiti,False,ali al hamadi
4,emerson palmieri dos santos,emerson palmieri,emerson palmieri,False,alisson
5,jhon duran,jader duran,jader duran,False,altay bayndr
6,andre trindade da costa neto,andreas pereira,andreas pereira,False,andre
7,richarlison de andrade,richarlison,richarlison,False,andreas pereira
8,francisco jorge tomas oliveira,matheus franca,matheus franca,False,antony
9,mitoma kaoru,kaoru mitoma,kaoru mitoma,False,armel bella kotchap


In [ ]:
import numpy as np
#load manually verified mappings
mapping_df = pd.read_csv("player_name_mapping_review.csv")
mapping_df['chosen_name'] = mapping_df.apply(lambda x: x['chosen_name'] if x['verified'] else np.nan, axis=1)
print(mapping_df.shape)
print(f'Out of {mapping_df.shape[0]} mappings, {mapping_df.verified.sum()} are being kept, rest thrown out!')

# Build mapping dicts (only verified rows will have values)
fpl_map = dict(zip(mapping_df['fpl_cleaned_name'], mapping_df['chosen_name']))
match_map = dict(zip(mapping_df['all_match_best_guess'], mapping_df['chosen_name']))

# Use the mapping *only where available*, else fallback to cleaned_name
fpl_df['mapped_Player'] = fpl_df['cleaned_name'].apply(
    lambda name: fpl_map.get(name, name)  # fallback to original cleaned name
)

all_match_df['mapped_Player'] = all_match_df['cleaned_Player'].apply(
    lambda name: match_map.get(name, name)
)

print(fpl_df.mapped_Player.isna().sum(), all_match_df.mapped_Player.isna().sum())
fpl_df = fpl_df.dropna(subset=['mapped_Player'])
all_match_df = all_match_df.dropna(subset=['mapped_Player'])

(109, 5)
Out of 109 mappings, 85 are being kept, rest thrown out!
871 341


In [ ]:
len(set(fpl_df['mapped_Player'].unique())), len(set(all_match_df['mapped_Player'].unique()))

(537, 547)

In [ ]:
fpl_only_cleaned = set(fpl_df['mapped_Player'].unique()) - set(all_match_df['mapped_Player'].unique())
all_match_only_cleaned = set(all_match_df['mapped_Player'].unique()) - set(fpl_df['mapped_Player'].unique())

print(f"\n{len(fpl_only_cleaned)} FPL names not in all_match_df")
print(f"{len(all_match_only_cleaned)} all_match_df names not in FPL")


14 FPL names not in all_match_df
24 all_match_df names not in FPL


In [ ]:
fpl_only_cleaned = set(fpl_df['match-name'].unique()) - set(all_match_df['simpler-match-name'].unique())
all_match_only_cleaned = set(all_match_df['simpler-match-name'].unique()) - set(fpl_df['match-name'].unique())

print(f"\n{len(fpl_only_cleaned)} FPL names not in all_match_df")
print(f"{len(all_match_only_cleaned)} all_match_df names not in FPL")


0 FPL names not in all_match_df
0 all_match_df names not in FPL


In [ ]:
# Create combined merge key in both dataframes
fpl_df['combined_key'] = (
    fpl_df['mapped_Player'].astype(str).str.strip()
    + " | "
    + fpl_df['match-name'].astype(str).str.strip()
)

all_match_df['combined_key'] = (
    all_match_df['mapped_Player'].astype(str).str.strip()
    + " | "
    + all_match_df['simpler-match-name'].astype(str).str.strip()
)

# Convert to sets
fpl_keys = set(fpl_df['combined_key'].dropna().unique())
allmatch_keys = set(all_match_df['combined_key'].dropna().unique())

# Differences
fpl_only = fpl_keys - allmatch_keys
allmatch_only = allmatch_keys - fpl_keys

# Report
print(f"\n{len(fpl_only)} combined keys in FPL missing from all_match_df")
print(f"{len(allmatch_only)} combined keys in all_match_df missing from FPL")



8555 combined keys in FPL missing from all_match_df
437 combined keys in all_match_df missing from FPL


/tmp/ipython-input-2878357826.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_match_df['combined_key'] = (


In [ ]:
fpl_df.columns

Index(['Unnamed: 0', 'name', 'position', 'team', 'xP', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW', 'fixture_code', 'fixture_event', 'fixture_finished',
       'fixture_finished_provisional', 'fixture_id', 'fixture_kickoff_time',
       'fixture_minutes', 'fixture_provisional_start_time', 'fixture_started',
       'fixture_team_a', 'fixture_team_a_score', 'fixture_team_h',
       'fixture_team_h_score', 'fixture_stats', 'fixture

In [ ]:
fpl_df[~fpl_df.combined_key.isin(fpl_only)].total_points.describe()

,total_points
count,10791.000000
mean,2.715874
std,2.910702
min,-5.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,25.000000


In [ ]:
fpl_df[fpl_df.combined_key.isin(fpl_only)].total_points.describe()

,total_points
count,8555.000000
mean,0.092928
std,0.673414
min,-2.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,17.000000


In [ ]:
# Merge on match-name and simpler-match-name AND player name
merged_df = pd.merge(
    fpl_df,
    all_match_df,
    left_on=['match-name', 'mapped_Player'],
    right_on=['simpler-match-name', 'mapped_Player'],
    how='outer',  # outer merge to see non-matching entries
    indicator=True  # adds column '_merge' to see merge status
)

# Show merge statistics
merge_stats = merged_df['_merge'].value_counts()
print("Merge stats:")
print(merge_stats)

# Players in fpl_df not in all_match_data
players_only_in_fpl = set(fpl_df['name']) - set(all_match_df['Player'])
print("\nPlayers only in FPL data:", len(players_only_in_fpl))

# Players in all_match_data not in fpl_df
players_only_in_all = set(all_match_df['Player']) - set(fpl_df['name'])
print("Players only in all_match_data:", len(players_only_in_all))

# Match names in fpl_df not in all_match_data (use simplified match names)
match_only_in_fpl = set(fpl_df['match-name']) - set(all_match_df['simpler-match-name'])
print("\nMatch names only in FPL data:", len(match_only_in_fpl))

# Match names in all_match_data not in fpl_df
match_only_in_all = set(all_match_df['simpler-match-name']) - set(fpl_df['match-name'])
print("Match names only in all_match_data:", len(match_only_in_all))


Merge stats:
_merge
both          10791
left_only      8555
right_only      437
Name: count, dtype: int64

Players only in FPL data: 96
Players only in all_match_data: 105

Match names only in FPL data: 0
Match names only in all_match_data: 0


In [ ]:
fpl_df.columns, all_match_df.columns

(Index(['Unnamed: 0', 'name', 'position', 'team', 'xP', 'assists', 'bonus',
        'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
        'expected_goal_involvements', 'expected_goals',
        'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
        'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
        'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
        'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
        'team_h_score', 'threat', 'total_points', 'transfers_balance',
        'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
        'GW', 'fixture_code', 'fixture_event', 'fixture_finished',
        'fixture_finished_provisional', 'fixture_id', 'fixture_kickoff_time',
        'fixture_minutes', 'fixture_provisional_start_time', 'fixture_started',
        'fixture_team_a', 'fixture_team_a_score', 'fixture_team_h',
        'fixture_team_h_score', 'fixture_st

In [ ]:
merged_df = pd.merge(
    fpl_df.drop(columns=[]),
    all_match_df.drop(columns=[]),
    left_on=['match-name', 'mapped_Player'],
    right_on=['simpler-match-name', 'mapped_Player'],
    how='inner',  # outer merge to see non-matching entries
    indicator=True  # adds column '_merge' to see merge status
)
print(merged_df._merge.value_counts())
droppers = ['Unnamed: 0_x', 'Unnamed: 0_y', 'combined_key_x', 'combined_key_y', 'cleaned_match_name_y', 'cleaned_match_name_x', 'cleaned_name', 'cleaned_Player', 'match-name_x', 'match-name_y', 'merge_player_name']
merged_df = merged_df.drop(columns=droppers+['_merge'])
# 1. Rename mapped_Player → name
merged_df = merged_df.rename(columns={'mapped_Player': 'mapping_name'})

# 2. Move "name" to the first column
cols = ['mapping_name'] + [c for c in merged_df.columns if c != 'mapping_name']
merged_df = merged_df[cols]

merged_df

_merge
both          10791
left_only         0
right_only        0
Name: count, dtype: int64


,mapping_name,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,...,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,simpler-match-name
0,alex scott,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,...,0,4,0,0,0,3,0,1,0.0,Nott'ham Forest vs Bournemouth
1,ben brereton,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,...,1,1,0,0,0,5,3,0,100.0,Newcastle Utd vs Southampton
2,pau torres,Pau Torres,DEF,Aston Villa,1.9,0,0,17,0,1.9,...,1,4,0,0,0,1,0,0,NaN,West Ham vs Aston Villa
3,hwang heechan,Hwang Hee-chan,MID,Wolves,1.3,0,0,14,0,16.3,...,1,0,0,0,0,4,1,1,50.0,Arsenal vs Wolves
4,joao gomes,João Victor Gomes da Silva,MID,Wolves,0.6,0,0,11,0,3.2,...,2,2,0,0,0,5,0,2,0.0,Arsenal vs Wolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10786,romain esse,Romain Esse,MID,Crystal Palace,1.8,0,0,3,0,1.4,...,0,0,0,0,0,2,0,0,NaN,Liverpool vs Crystal Palace
10787,raheem sterling,Raheem Sterling,MID,Arsenal,2.3,0,0,10,0,7.1,...,2,0,0,0,0,2,0,0,NaN,Southampton vs Arsenal
10788,raul jimenez,Raúl Jiménez,FWD,Fulham,5.0,0,0,1,0,0.6,...,0,0,0,0,0,2,4,1,80.0,Fulham vs Manchester City
10789,myles lewisskelly,Myles Lewis-Skelly,MID,Arsenal,3.3,0,0,5,0,1.3,...,0,0,0,0,0,2,1,0,100.0,Southampton vs Arsenal


In [ ]:
merged_df.total_points.sum()

np.int64(29307)

In [ ]:
merged_df.to_csv('merged_fpl_dataset.csv')

In [ ]:
merged_df['mapped_Player']

,mapped_Player
0,alex scott
1,ben brereton
2,pau torres
3,hwang heechan
4,joao gomes
...,...
10786,romain esse
10787,raheem sterling
10788,raul jimenez
10789,myles lewisskelly


In [ ]:
import pandas as pd

all_match_df = pd.read_csv('/content/all_match_data.csv')
fixtures_df = pd.read_csv('/content/fixtures.csv')
fpl_df = pd.read_csv('/content/fpl_data.csv')

In [ ]:
fixtures_df[fixtures_df.Wk == 15]

,Unnamed: 0,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,match-name
154,154,15.0,Sat,2024-12-07,15:00,Aston Villa,2.3,1–0,0.3,Southampton,"42,453",Villa Park,Darren Bond,Match Report,NaN,Week-15 Aston Villa vs Southampton
155,155,15.0,Sat,2024-12-07,15:00,Crystal Palace,1.3,2–2,1.4,Manchester City,"25,142",Selhurst Park,Robert Jones,Match Report,NaN,Week-15 Crystal Palace vs Manchester City
156,156,15.0,Sat,2024-12-07,15:00,Brentford,1.2,4–2,1.5,Newcastle Utd,"17,078",Gtech Community Stadium,Simon Hooper,Match Report,NaN,Week-15 Brentford vs Newcastle Utd
157,157,15.0,Sat,2024-12-07,17:30,Manchester Utd,1.6,2–3,0.8,Nott'ham Forest,"73,778",Old Trafford,Darren England,Match Report,NaN,Week-15 Manchester Utd vs Nott'ham Forest
158,158,15.0,Sun,2024-12-08,14:00,Leicester City,1.3,2–2,1.5,Brighton,"31,647",King Power Stadium,Stuart Attwell,Match Report,NaN,Week-15 Leicester City vs Brighton
159,159,15.0,Sun,2024-12-08,14:00,Ipswich Town,1.3,1–2,3.2,Bournemouth,"29,180",Portman Road Stadium,Michael Salisbury,Match Report,NaN,Week-15 Ipswich Town vs Bournemouth
160,160,15.0,Sun,2024-12-08,14:00,Fulham,0.2,1–1,1.8,Arsenal,"26,954",Craven Cottage,Chris Kavanagh,Match Report,NaN,Week-15 Fulham vs Arsenal
161,161,15.0,Sun,2024-12-08,16:30,Tottenham,2.8,3–4,2.8,Chelsea,"61,184",Tottenham Hotspur Stadium,Anthony Taylor,Match Report,NaN,Week-15 Tottenham vs Chelsea
162,162,15.0,Mon,2024-12-09,20:00,West Ham,1.0,2–1,1.4,Wolves,"62,474",London Stadium,John Brooks,Match Report,NaN,Week-15 West Ham vs Wolves
263,263,15.0,Wed,2025-02-12,19:30,Everton,1.0,2–2,0.6,Liverpool,"39,280",Goodison Park,Michael Oliver,Match Report,NaN,Week-15 Everton vs Liverpool


In [ ]:
fpl_df['match-name']

,match-name
0,Week-1 West Ham vs Aston Villa
1,Week-1 West Ham vs Aston Villa
2,Week-1 Newcastle Utd vs Southampton
3,Week-1 Chelsea vs Manchester City
4,Week-1 Arsenal vs Wolves
...,...
14173,Week-19 Crystal Palace vs Southampton
14174,Week-20 Bournemouth vs Everton
14175,Week-19 Ipswich Town vs Chelsea
14176,Week-19 Aston Villa vs Brighton


In [ ]:
fpl_df.columns

Index(['Unnamed: 0', 'name', 'position', 'team', 'xP', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW', 'match-name'],
      dtype='object')

In [ ]:
#filter based on total_points and maybe minutes
(fpl_df[['name', 'minutes']].groupby(by=['name'])['minutes'].sum())

,minutes
name,
Aaron Cresswell,59
Aaron Hickey,0
Aaron Ramsdale,1260
Aaron Wan-Bissaka,1714
Abdoulaye Doucouré,1424
...,...
Zach Marsh,0
Álex Moreno Lopera,788
Ângelo Gabriel Borges Damaceno,0


In [ ]:
fpl_df[['name', 'match-name', 'bonus', 'total_points']]

,name,match-name,bonus,total_points
0,Alex Scott,Week-1 West Ham vs Aston Villa,0,2
1,Carlos Miguel dos Santos Pereira,Week-1 West Ham vs Aston Villa,0,0
2,Tomiyasu Takehiro,Week-1 Newcastle Utd vs Southampton,0,0
3,Malcolm Ebiowei,Week-1 Chelsea vs Manchester City,0,0
4,Ben Brereton Díaz,Week-1 Arsenal vs Wolves,0,1
...,...,...,...,...
14173,Giovani Lo Celso,Week-19 Crystal Palace vs Southampton,0,0
14174,Tawanda Chirewa,Week-20 Bournemouth vs Everton,0,0
14175,Cameron Humphreys,Week-19 Ipswich Town vs Chelsea,0,0
14176,Ashley Young,Week-19 Aston Villa vs Brighton,0,2


In [ ]:
all_match_df.columns, fpl_df.columns

(Index(['Unnamed: 0', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min',
        'Performance_Gls', 'Performance_Ast', 'Performance_PK',
        ...
        'Performance_Crs', 'Performance_TklW', 'Performance_PKwon',
        'Performance_PKcon', 'Performance_OG', 'Performance_Recov',
        'Aerial Duels_Won', 'Aerial Duels_Lost', 'Aerial Duels_Won%',
        'match-name'],
       dtype='object', length=117),
 Index(['Unnamed: 0', 'name', 'position', 'team', 'xP', 'assists', 'bonus',
        'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
        'expected_goal_involvements', 'expected_goals',
        'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
        'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
        'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
        'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
        'team_h_score', 'threat', 'total_points', 'transfers_balance',
     

In [ ]:
display(all_match_df.head())
display(all_match_df.columns)
display(fpl_df.head())
display(fpl_df.columns)

,Unnamed: 0,Player,#,Nation,Pos,Age,Min,Performance_Gls,Performance_Ast,Performance_PK,...,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,match-name
0,0,Noussair Mazraoui,3,ma MAR,RB,26-276,80,0,0,0,...,0,2,0,0,0,5,3,1,75.0,Week-1 Manchester Utd vs Fulham
1,1,Matthijs de Ligt,4,nl NED,CB,25-004,10,0,0,0,...,0,0,0,0,0,0,0,1,0.0,Week-1 Manchester Utd vs Fulham
2,2,Harry Maguire,5,eng ENG,CB,31-164,80,0,0,0,...,0,0,0,0,0,5,5,1,83.3,Week-1 Manchester Utd vs Fulham
3,3,Lisandro Martínez,6,ar ARG,"CB,LB",26-211,90,0,0,0,...,0,2,0,0,0,2,0,1,0.0,Week-1 Manchester Utd vs Fulham
4,4,Mason Mount,7,eng ENG,AM,25-219,60,0,0,0,...,0,1,0,0,0,1,1,0,100.0,Week-1 Manchester Utd vs Fulham


Index(['Unnamed: 0', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min',
       'Performance_Gls', 'Performance_Ast', 'Performance_PK',
       ...
       'Performance_Crs', 'Performance_TklW', 'Performance_PKwon',
       'Performance_PKcon', 'Performance_OG', 'Performance_Recov',
       'Aerial Duels_Won', 'Aerial Duels_Lost', 'Aerial Duels_Won%',
       'match-name'],
      dtype='object', length=117)

,Unnamed: 0,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,match-name
0,0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,...,0.0,2,0,0,0,50,False,0,1,Week-1 West Ham vs Aston Villa
1,1,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,2.2,0,0,0,0,0.0,...,0.0,0,0,0,0,45,True,0,1,Week-1 West Ham vs Aston Villa
2,2,Tomiyasu Takehiro,DEF,Arsenal,0.0,0,0,0,0,0.0,...,0.0,0,0,0,0,50,True,0,1,Week-1 Newcastle Utd vs Southampton
3,3,Malcolm Ebiowei,MID,Crystal Palace,0.0,0,0,0,0,0.0,...,0.0,0,0,0,0,45,False,0,1,Week-1 Chelsea vs Manchester City
4,4,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,...,16.0,1,0,0,0,55,False,1,1,Week-1 Arsenal vs Wolves


Index(['Unnamed: 0', 'name', 'position', 'team', 'xP', 'assists', 'bonus',
       'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW', 'match-name'],
      dtype='object')

In [ ]:
a = fpl_df['name'] + fpl_df['match-name']
b = all_match_df['Player'] + all_match_df['match-name']
len(set(a).intersection(set(b))), len(set(a)), len(set(b))

(564, 13430, 11567)

In [ ]:
a = fpl_df['name']
b = all_match_df['Player']
len(set(a).intersection(set(b))), len(set(a)), len(set(b))

(417, 724, 562)

In [ ]:
a = fpl_df['match-name']
b = all_match_df['match-name']
len(set(a).intersection(set(b)))

190

In [ ]:
merged_df = pd.merge(fpl_df, all_match_df, left_on=['match-name', 'name'], right_on=['match-name', 'Player'], how='left', indicator=True)
display(merged_df.head())

,Unnamed: 0_x,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,...,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,_merge
0,0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,1,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,2.2,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,2,Tomiyasu Takehiro,DEF,Arsenal,0.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,3,Malcolm Ebiowei,MID,Crystal Palace,0.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,4,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
merged_df._merge.value_counts()

,count
_merge,
left_only,13614
both,564
right_only,0


In [ ]:
%pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.7 MB/s eta 0:00:00


In [ ]:
import re
from rapidfuzz import process, fuzz
import unicodedata

def clean_name(name):
    if isinstance(name, str):
        # Normalize unicode characters (é -> e, ü -> u, ñ -> n, etc.)
        name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
        # Remove remaining special characters
        name = re.sub(r'[^a-zA-Z0-9\s]', '', name)
        # Convert to lowercase and strip whitespace
        name = name.lower().strip()
    return name

# Apply cleaning to player names in both dataframes
fpl_df['cleaned_name'] = fpl_df['name'].apply(clean_name)
all_match_df['cleaned_Player'] = all_match_df['Player'].apply(clean_name)

# Identify unique cleaned names that are mismatched
fpl_only_cleaned = set(fpl_df['cleaned_name'].unique()) - set(all_match_df['cleaned_Player'].unique())
all_match_only_cleaned = set(all_match_df['cleaned_Player'].unique()) - set(fpl_df['cleaned_name'].unique())

print(f"\n{len(fpl_only_cleaned)} Cleaned player names in fpl_df but not in all_match_df (sample): {list(fpl_only_cleaned)[:20]}")
print(f"\n{len(all_match_only_cleaned)} Cleaned player names in all_match_df but not in fpl_df (sample): {list(all_match_only_cleaned)[:20]}")


# Function to generate possible name variations (normal and reversed)
def name_variations(name):
    parts = name.split()
    if len(parts) > 1:
        reversed_name = " ".join(parts[::-1])
        return [name, reversed_name]
    else:
        return [name]

# Create a mapping using fuzzy matching, considering reversed names
player_name_mapping = {}
for fpl_name in fpl_only_cleaned:
    best_score = 0
    best_match = None
    for candidate in all_match_only_cleaned:
        for variant in name_variations(fpl_name):
            score = fuzz.token_sort_ratio(variant, candidate)
            if score > best_score:
                best_score = score
                best_match = candidate
    if best_score > 80:  # Threshold for matching
        player_name_mapping[fpl_name] = best_match

print(f"\nCreated a mapping for {len(player_name_mapping)} player names.")
# Display a sample of the created mapping
print("Sample player name mapping:")
display(list(player_name_mapping.items())[:20])


# Create a new column in fpl_df with the mapped player name from all_match_df
fpl_df['mapped_Player'] = fpl_df['cleaned_name'].map(player_name_mapping)

# Combine the original cleaned name with the mapped name for merging
# If a name was not in the fpl_only_cleaned set, use its own cleaned name
fpl_df['merge_player_name'] = fpl_df['cleaned_name'].apply(lambda x: player_name_mapping.get(x, x))

# Now try merging again using the cleaned and potentially mapped player names and match-name
# Clean and standardize 'match-name' in both dataframes
def clean_match_name(name):
    if isinstance(name, str):
        name = name.strip()
    return name

fpl_df['cleaned_match_name'] = fpl_df['match-name'].apply(clean_match_name)
all_match_df['cleaned_match_name'] = all_match_df['match-name'].apply(clean_match_name)


merged_df_cleaned = pd.merge(
    fpl_df,
    all_match_df,
    left_on=['cleaned_match_name', 'merge_player_name'],
    right_on=['cleaned_match_name', 'cleaned_Player'],
    how='left'
)

display(merged_df_cleaned.head())

print("\nChecking for NaNs in columns from all_match_df after cleaning and mapping:")
# Select a few representative columns from all_match_df to check for NaNs
all_match_cols_to_check_cleaned = ['Player', 'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Lost', 'Aerial Duels_Won%']
print(merged_df_cleaned[all_match_cols_to_check_cleaned].isnull().sum())


297 Cleaned player names in fpl_df but not in all_match_df (sample): ['luis guilherme lira dos santos', 'david datro fofana', 'andrew moran', 'alfie gilchrist', 'tom cannon', 'shea charles', 'zach marsh', 'jamaldeen jimoh', 'welington damascena santos', 'luke cundle', 'lino da cruz sousa', 'samuel raksakyi', 'francisco evanilson de lima barbosa', 'sugawara yukinari', 'chadi riad dnanou', 'matheus santos carneiro da cunha', 'toti antonio gomes', 'kurt zouma', 'edson alvarez velazquez', 'malachi hardy']

135 Cleaned player names in all_match_df but not in fpl_df (sample): ['ruben dias', 'sammie szmodics', 'mateus fernandes', 'mateus mane', 'joelinton', 'marc guiu', 'daniel podence', 'pedro neto', 'joao felix', 'chidozie obimartin', 'nasser djiga', 'darwin nunez', 'goncalo guedes', 'morato', 'olabade aluko', 'kaoru mitoma', 'takehiro tomiyasu', 'oliver scarles', 'diogo dalot', 'valentino livramento']

Created a mapping for 27 player names.
Sample player name mapping:


[('sugawara yukinari', 'yukinari sugawara'),
 ('konstantinos tsimikas', 'kostas tsimikas'),
 ('victor kristiansen', 'victor bernth kristiansen'),
 ('bruno borges fernandes', 'bruno fernandes'),
 ('will smallbone', 'william smallbone'),
 ('sam szmodics', 'sammie szmodics'),
 ('gabriel martinelli silva', 'gabriel martinelli'),
 ('ben brereton diaz', 'ben brereton'),
 ('tino livramento', 'valentino livramento'),
 ('jaden philogene', 'jaden philogene bidace'),
 ('endo wataru', 'wataru endo'),
 ('josh king', 'joshua king'),
 ('emiliano martinez romero', 'emiliano martinez'),
 ('vitalii mykolenko', 'vitaliy mykolenko'),
 ('marc guiu paz', 'marc guiu'),
 ('carlos alcaraz duran', 'carlos alcaraz'),
 ('jefferson lerma solis', 'jefferson lerma'),
 ('matheus luiz nunes', 'matheus nunes'),
 ('tomiyasu takehiro', 'takehiro tomiyasu'),
 ('altay bayindir', 'altay bayndr')]

,Unnamed: 0_x,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,...,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,match-name_y,cleaned_Player
0,0,Alex Scott,MID,Bournemouth,1.6,0,0,11,0,12.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Carlos Miguel dos Santos Pereira,GK,Nott'm Forest,2.2,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Tomiyasu Takehiro,DEF,Arsenal,0.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Malcolm Ebiowei,MID,Crystal Palace,0.0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Ben Brereton Díaz,MID,Southampton,1.0,0,0,-2,0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Checking for NaNs in columns from all_match_df after cleaning and mapping:
Player               13568
Performance_Recov    13568
Aerial Duels_Won     13568
Aerial Duels_Lost    13568
Aerial Duels_Won%    13770
dtype: int64


In [ ]:
list(.columns)

['Unnamed: 0',
 'name',
 'position',
 'team',
 'xP',
 'assists',
 'bonus',
 'bps',
 'clean_sheets',
 'creativity',
 'element',
 'expected_assists',
 'expected_goal_involvements',
 'expected_goals',
 'expected_goals_conceded',
 'fixture',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'kickoff_time',
 'minutes',
 'modified',
 'opponent_team',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'round',
 'saves',
 'selected',
 'starts',
 'team_a_score',
 'team_h_score',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'was_home',
 'yellow_cards',
 'GW',
 'match-name',
 'cleaned_name',
 'mapped_Player',
 'merge_player_name',
 'cleaned_match_name']

In [ ]:
fpl_df[fpl_df.cleaned_name.isin(fpl_only_cleaned) & fpl_df.mapped_Player.isnull()].cleaned_name.unique().shape

(270,)

In [ ]:
import re

week = all_match_df["match-name"].apply(lambda s: re.search(r"Week-(\d+)", s).group(1))
home = all_match_df["match-name"].apply(lambda s: re.search(r"Week-\d+\s+(.*?)\s+vs", s).group(1))
away = all_match_df["match-name"].apply(lambda s: re.search(r"vs\s+(.*)", s).group(1))
home.unique(), away.unique(), week.unique()

(array(['Manchester Utd', 'Ipswich Town', 'Newcastle Utd', 'Everton',
        "Nott'ham Forest", 'Arsenal', 'West Ham', 'Brentford', 'Chelsea',
        'Leicester City', 'Brighton', 'Manchester City', 'Southampton',
        'Tottenham', 'Fulham', 'Crystal Palace', 'Aston Villa', 'Wolves',
        'Bournemouth', 'Liverpool'], dtype=object),
 array(['Fulham', 'Liverpool', 'Southampton', 'Brighton', 'Bournemouth',
        'Wolves', 'Aston Villa', 'Crystal Palace', 'Manchester City',
        'Tottenham', 'Manchester Utd', 'Ipswich Town', "Nott'ham Forest",
        'Everton', 'Leicester City', 'West Ham', 'Arsenal', 'Chelsea',
        'Newcastle Utd', 'Brentford'], dtype=object),
 array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
        '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
        '24', '25', '29', '26', '27', '28', '30', '31', '32', '33', '34',
        '35', '36', '37', '38'], dtype=object))

In [ ]:
fpl_df["match-name"].value_counts()

,count
match-name,
Week- vs,1300
Week-17 Leicester City vs Wolves,85
Week-8 Wolves vs Manchester City,84
Week-12 Aston Villa vs Crystal Palace,84
Week-16 Bournemouth vs West Ham,82
...,...
Week-4 Liverpool vs Nott'ham Forest,56
Week-7 Manchester City vs Fulham,56
Week-1 Ipswich Town vs Liverpool,55


In [ ]:
# Check the number of unique match names in each DataFrame
print(f"Unique match names in fpl_df: {fpl_df['match-name'].nunique()}")
print(f"Unique match names in all_match_df: {all_match_df['match-name'].nunique()}")

# Find match names in fpl_df that are not in all_match_df
fpl_only_matches = set(fpl_df['match-name'].unique()) - set(all_match_df['match-name'].unique())
print(f"\nMatch names in fpl_df but not in all_match_df: {fpl_only_matches}")

# Find match names in all_match_df that are not in fpl_df
all_match_only_matches = set(all_match_df['match-name'].unique()) - set(fpl_df['match-name'].unique())
print(f"\nMatch names in all_match_df but not in fpl_df: {all_match_only_matches}")

Unique match names in fpl_df: 191
Unique match names in all_match_df: 380

Match names in fpl_df but not in all_match_df: {'Week-  vs '}

Match names in all_match_df but not in fpl_df: {'Week-21 Brentford vs Manchester City', 'Week-26 Southampton vs Brighton', "Week-33 Tottenham vs Nott'ham Forest", 'Week-36 Wolves vs Brighton', 'Week-21 Newcastle Utd vs Wolves', 'Week-28 Brighton vs Fulham', "Week-23 Bournemouth vs Nott'ham Forest", 'Week-22 Manchester Utd vs Brighton', 'Week-33 Manchester Utd vs Wolves', 'Week-38 Fulham vs Manchester City', 'Week-37 Manchester City vs Bournemouth', 'Week-38 Southampton vs Arsenal', 'Week-34 Arsenal vs Crystal Palace', 'Week-20 Southampton vs Brentford', 'Week-38 Newcastle Utd vs Everton', 'Week-25 Liverpool vs Wolves', 'Week-30 Wolves vs West Ham', 'Week-37 Leicester City vs Ipswich Town', 'Week-34 Bournemouth vs Manchester Utd', 'Week-21 Leicester City vs Crystal Palace', 'Week-24 Manchester Utd vs Crystal Palace', 'Week-29 Manchester City vs Bright

In [ ]:
print(f"Number of rows in fpl_df: {fpl_df.shape[0]}")
print(f"Number of rows in merged_df: {merged_df.shape[0]}")

print("\nColumns in merged_df:")
print(merged_df.columns.tolist())

print("\nExamining rows with expected successful merges:")
# Select a few rows where we expect a match based on the original data inspection.
# For example, a known player and match from the head of all_match_df
expected_match_rows = merged_df[
    (merged_df['match-name'] == 'Week-1 Manchester Utd vs Fulham') &
    (merged_df['name'] == 'Noussair Mazraoui')
]
display(expected_match_rows)

print("\nChecking for NaNs in columns from all_match_df:")
# Select a few representative columns from all_match_df to check for NaNs
all_match_cols_to_check = ['Player', 'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Lost', 'Aerial Duels_Won%']
print(merged_df[all_match_cols_to_check].isnull().sum())

Number of rows in fpl_df: 14178
Number of rows in merged_df: 14178

Columns in merged_df:
['Unnamed: 0_x', 'name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes', 'modified', 'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score', 'threat', 'total_points', 'transfers_balance', 'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW', 'match-name', 'Unnamed: 0_y', 'Player', '#', 'Nation', 'Pos', 'Age', 'Min', 'Performance_Gls', 'Performance_Ast', 'Performance_PK', 'Performance_PKatt', 'Performance_Sh', 'Performance_SoT', 'Performance_CrdY', 'Performance_CrdR', 'Performance_Touches', 'Performance_Tkl', 'Performance_Int', 'Performance_Block

,Unnamed: 0_x,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,...,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%



Checking for NaNs in columns from all_match_df:
Player               13614
Performance_Recov    13614
Aerial Duels_Won     13614
Aerial Duels_Lost    13614
Aerial Duels_Won%    13801
dtype: int64


In [ ]:
# Sample some player names from each DataFrame
print("Sample player names from all_match_df:")
display(all_match_df['Player'].sample(10))

print("\nSample player names from fpl_df:")
display(fpl_df['name'].sample(10))

# Find player names in fpl_df that are not in all_match_df
fpl_only_players = set(fpl_df['name'].str.lower().unique()) - set(all_match_df['Player'].str.lower().unique())
print(f"\n{len(fpl_only_players)} Player names in fpl_df but not in all_match_df (sample): {list(fpl_only_players)[:20]}") # Displaying a sample of mismatches

# Find player names in all_match_df that are not in fpl_df
all_match_only_players = set(all_match_df['Player'].str.lower().unique()) - set(fpl_df['name'].str.lower().unique())
print(f"\n{len(all_match_only_players)} Player names in all_match_df but not in fpl_df (sample): {list(all_match_only_players)[:20]}") # Displaying a sample of mismatches

Sample player names from all_match_df:


,Player
6449,Vitaly Janelt
685,Morgan Gibbs-White
3094,Asher Agbinone
10937,Kieran Tierney
8877,Christian Nørgaard
10699,Vitaliy Mykolenko
6912,Emerson Palmieri
5404,Jan Paul van Hecke
4577,Mario Lemina
10059,Emiliano Martínez



Sample player names from fpl_df:


,name
7574,Freddie Ladapo
12282,Noussair Mazraoui
13188,Lewis Orford
2241,Salah-Eddine Oulad M'hand
11278,Cameron Burgess
11922,John Stones
9162,Sandro Tonali
1174,Timo Werner
3340,Ethan Brierley
3872,Cameron Humphreys



306 Player names in fpl_df but not in all_match_df (sample): ['luis guilherme lira dos santos', 'david datro fofana', 'andrew moran', 'alejo véliz', 'joelinton cássio apolinário de lira', 'tom cannon', 'shea charles', 'jamaldeen jimoh', 'zach marsh', 'alfie gilchrist', 'welington damascena santos', 'luke cundle', 'sékou mara', 'lino da cruz sousa', 'francisco evanilson de lima barbosa', 'armel bella-kotchap', 'sugawara yukinari', 'chadi riad dnanou', 'matheus santos carneiro da cunha', 'min-hyeok yang']

144 Player names in all_match_df but not in fpl_df (sample): ['sammie szmodics', 'mateus fernandes', 'mateus mane', 'gabriel magalhães', 'joelinton', 'marc guiu', 'gonçalo guedes', 'daniel podence', 'pedro neto', 'miguel almirón', 'nasser djiga', 'albert grønbaek', 'caoimhín kelleher', 'morato', 'antonín kinský', 'olabade aluko', 'álex moreno', 'kaoru mitoma', 'takehiro tomiyasu', 'oliver scarles']
